# Day-49: Time Series Forecasting Projet

## Step-1 ENV setup

In [ ]:
import pandas as pd
import numpy as np

## Step-2 Dataset Loading

In [ ]:
df = pd.read_csv('BKB_WaterQualityData_2020084.csv')
df.head()

In [ ]:
df.shape

## Step-3 EDA and Data Cleaning

In [ ]:
df.info()

### Site Id

In [ ]:
df[df['Site_Id'].isnull()]

In [ ]:
df['Site_Id'].value_counts()

In [ ]:
df[df['Site_Id'] == 'd']

#### Correcting Case error from Site_Id

In [ ]:
df['Site_Id'] = df['Site_Id'].str.upper()

In [ ]:
df['Site_Id'].value_counts()

In [ ]:
df[df['Site_Id'].isnull()]

### Droping invalid Site_Id entries

In [ ]:
df.dropna(subset=['Site_Id'], inplace=True)

In [ ]:
df.isnull().sum()

### Unit Id

In [ ]:
df['Unit_Id'].value_counts()

unit_id consist of only 1 type of data points with typo error and also not for all data points which we can drop it.

In [ ]:
df[df['Read_Date'].isnull()]

#### WhoVerified

In [ ]:
df['WhoVerified'].value_counts()

we can drop WhoVerified column as its are not useful for our analysis

### Field_Tech

In [ ]:
df['Field_Tech'].value_counts()

we can drop Field_Tech column as its are not useful for our analysis

### Salinity (ppt)

In [ ]:
df[df['Salinity (ppt)'].isna()].shape

In [ ]:
salinity = pd.to_numeric(df['Salinity (ppt)'], errors='coerce')
salinity.describe()

In [ ]:
salinity.hist(bins=50)

In [ ]:
s = pd.to_numeric(df['Salinity (ppt)'], errors='coerce')
miss_rate = s.isna().mean()
by_site = df.groupby('Site_Id')['Salinity (ppt)'].apply(lambda x: x.isna().mean()).sort_values(ascending=False)
print("Overall missing rate:", round(100*miss_rate,2), "%")
print(by_site.head(10))

In [ ]:
salinity.isna().sum()

#### Air Temprature

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

tmp = df.copy()
tmp['Read_Date'] = pd.to_datetime(tmp['Read_Date'], errors='coerce')
tmp = tmp.sort_values('Read_Date')

s = pd.to_numeric(tmp['AirTemp (C)'], errors='coerce')

plt.figure()
plt.plot(tmp['Read_Date'], s, marker='o', linestyle='-')
plt.title('AirTemp (C) over time')
plt.xlabel('Read_Date')
plt.ylabel('AirTemp (C)')
plt.tight_layout()
plt.show()
plt.close()

In [ ]:
import numpy as np
import pandas as pd

# Parse both columns as numbers
air_temp_cel = pd.to_numeric(df['Air Temp-Celsius'], errors='coerce')   # supposed Celsius, but may contain °F
air_temp_c = pd.to_numeric(df['AirTemp (C)'], errors='coerce')        # Celsius
air_temp_f = pd.to_numeric(df['Air Temp (?F)'], errors='coerce')        # Fahrenheit

# Helper: treat rows where both are present
both = air_temp_cel.notna() & air_temp_c.notna()

# 1) Rows where the two columns disagree (as-entered)
mismatch = both & ~np.isclose(air_temp_cel, air_temp_c, atol=0.5)

mismatch_count = mismatch.sum()
print(f"Number of rows where both columns are present but disagree: {mismatch_count}")

df[mismatch][['Air Temp-Celsius','Air Temp (?F)', 'AirTemp (C)']].head()


In [ ]:
df[mismatch][['Air Temp-Celsius','Air Temp (?F)','Water Temp (?C)', 'AirTemp (C)']]

In [ ]:
negative_temp = df['AirTemp (C)'] <= 0

negative_temp_count = negative_temp.sum()
print(f"Number of rows where 'AirTemp (C)' is less than or equal to 0: {negative_temp_count}")

df[negative_temp][['Air Temp-Celsius','Air Temp (?F)','Water Temp (?C)', 'AirTemp (C)']]

In [ ]:
df[negative_temp & both]

#### comparing air_temp in c and air_temp in f

In [ ]:
convert_f_to_c = (air_temp_f - 32.0) * (5.0/9.0)

# Helper: treat rows where both are present
both = convert_f_to_c.notna() & air_temp_c.notna()

mismatch_convert_temp = both & ~np.isclose(convert_f_to_c, air_temp_c, rtol=0, atol=0.5)

mismatch_convert_temp_count = mismatch_convert_temp.sum()
match_count = (both & ~mismatch_convert_temp).sum()

print(f"Rows with both F and C present: {both.sum()}")
print(f"Matches (F→C ≈ AirTemp (C)): {match_count}")
print(f"Mismatches: {mismatch_convert_temp_count}")

In [ ]:
# Define buckets (°C) and labels
bins    = [-np.inf, -10, 5, 25, 35, np.inf]   # [-inf,-10), [-10,5), [5,25), [25,35), [35,inf)
labels  = ['extreme cold', 'cold', 'normal', 'hot', 'extreme hot']

# Categorize (right=False means left-inclusive, right-exclusive)
cats = pd.cut(air_temp_c, bins=bins, labels=labels, right=False)

# Overall counts (ordered)
counts = cats.value_counts().reindex(labels, fill_value=0)
print(counts)

In [ ]:
# just extreme cold
extreme_cold_temp = df.loc[cats == 'extreme cold']

# extreme cold AND 'Air Temp-Celsius' is not null
extreme_cold_temp = df.loc[(cats == 'extreme cold') & (air_temp_cel.notna())]
extreme_cold_temp[['Air Temp-Celsius','Air Temp (?F)','Water Temp (?C)', 'AirTemp (C)']]

In [ ]:
# just extreme cold
extreme_cold_temp = df.loc[cats == 'extreme cold']

# extreme cold AND 'Air Temp-Celsius' is not null
extreme_cold_temp_f = df.loc[(cats == 'extreme cold') & (air_temp_f.notna())]
extreme_cold_temp_f[['Air Temp-Celsius','Air Temp (?F)','Water Temp (?C)', 'AirTemp (C)']]

In [ ]:
# just extreme cold
extreme_cold_temp = df.loc[cats == 'extreme cold']

# extreme cold AND 'Air Temp-Celsius' is not null
extreme_cold_temp_na = df.loc[(cats == 'extreme cold') & (air_temp_f.isna())]
print(extreme_cold_temp_na.count().sum())
avg_extreme_cold_temp_na = pd.to_numeric(extreme_cold_temp_na['AirTemp (C)'], errors='coerce')
print(avg_extreme_cold_temp_na.mean())

extreme_cold_temp_na[['Air Temp-Celsius','Air Temp (?F)','Water Temp (?C)', 'AirTemp (C)']]

In [ ]:
df.drop(columns=['Unit_Id','Air Temp-Celsius','Air Temp (?F)','WhoVerified','Field_Tech'], inplace=True)

In [ ]:
df.isnull().sum()

In [ ]:
df[df['Read_Date'].isnull()]

In [ ]:
df['Read_Date'].dropna(inplace=True)

In [ ]:
df.dropna(subset=['Read_Date'], inplace=True)

In [ ]:
df.isnull().sum()